> Instalación de dependencias

In [ ]:
%conda install pip
%conda install -c conda-forge selenium

# Instalación del geckodriver
%pip install webdriver_manager

In [1]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\daniel.martinezcarre\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import pprint
import numpy as np


In [78]:
driver = webdriver.Firefox()

In [79]:
def esperar_carga_pagina(driver, url):
    """
    Espera a que la página web se cargue completamente.

    Args:
        driver: El objeto del navegador web.
        url: La URL de la página web que se quiere cargar.
    """
    # Carga la página web
    driver.get(url)

    # Espera a que la página se cargue completamente
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")

    # Filtrado de "All datasets"
    btn_all = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/main/div[1]/div/div[5]/div[2]/div[4]/div/div/div[2]/button[1]")))
    btn_all.click()

    # Filtrado por "Most Votes"
    filtrado = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".sc-edDLmV > div:nth-child(1) > button:nth-child(2)")))
    filtrado.click()
    most_votes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.mdc-menu:nth-child(3) > ul:nth-child(1) > li:nth-child(2)")))
    most_votes.click()

    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")

### Sacar enlaces de datasets de una página

In [80]:
def get_links_datasets_page():
    urls = []
    lis = driver.find_elements(By.XPATH, './/ul[@class="km-list km-list--three-line"]/li')
    for i in lis:
        enlace = i.find_element(By.XPATH, './div/a').get_attribute("href")
        urls.append(enlace)
    return urls


In [81]:
URL = "https://www.kaggle.com"
esperar_carga_pagina(driver, URL+"/datasets")

In [82]:
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".sc-hXOTHz")))
links_datasets = get_links_datasets_page()

### Función para pasar de página

In [83]:
# def next_page():
    

In [84]:
def get_info_dataset(url):

    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")

    row = {}
    tag_list = []
    body = driver.execute_script("return document.body")
    source = body.get_attribute('innerHTML')
    soup = BeautifulSoup(source, 'html.parser')

    # NAME
    name = soup.find("h1", class_="sc-eTNRI sc-cMAQNS iudEzZ hVdeow")
    row["name"] = name.text

    # URL
    row["url"] = url

    # DESCRIPTION
    try:
        description = soup.find("span", class_="sc-dmXWDj sc-bNSORF cLdaqZ cCQcWJ")
        row["description"] = description.text
    except:
        row["description"] = np.nan

    # AUTHOR
    try:
        author = soup.find("span", class_="sc-lnsjTu gQDhUC")
        row["author"] = author.text
    except:
        row["author"] = np.nan
    # VOTES
    try:
        votes = soup.find("button", class_="sc-hSWyVn sc-eFzpJt iJoZbm hPXpRk")
        row["votes"] = votes.text
    except:
        row["votes"] = np.nan

    # SIZE
    try:
        size = soup.find("span", class_="sc-fmKFGs sc-htdKJd hXhfOa dxMHrx")
        row["size"] = size.text
    except:
        row["size"] = np.nan

    # Expected update frequency
    try:
        update_frequency = soup.find_all("p", class_="sc-dJGMql sc-lkDZZF jCrglO cwGkEk")
        row["update_frequency"] = update_frequency[1].text
    except:
        row["update_frequency"] = np.nan

    # LICENSE
    try:
        row["license"] = update_frequency[0].text
    except:
        row["license"] = np.nan
        
    # USABILITY (css_selector)
    try:
        usability = soup.select_one(".sc-dTbyjA > p:nth-child(2) > span:nth-child(1)")
        row["usability"] = usability.text
    except:
        row["usability"] = np.nan

    # TAGS
    try:
        tags = soup.find("div", id="combo-tags-menu-chipset")
        for i in tags:
            tag_list.append(i.text)
        row["tags"] = tag_list
    except:
        row["tags"] = np.nan

    # Number of files
    try:
        files = soup.findAll("p", class_="sc-dJGMql sc-hA-dcfU jCrglO eynbfF")
        row["files"] = files[0].text
    except:
        row["files"] = np.nan

    # Number of columns
    try:
        row["columns"] = files[1].text
    except:
        row["columns"] = np.nan

    # Views
    try:
        views = soup.findAll("h5", class_="sc-fmKFGs sc-cUPRRX hXhfOa llGCIX")
        row["views"] = views[0].text
    except:
        row["views"] = np.nan

    # Downloads
    try:
        row["downloads"] = views[1].text
    except:
        row["downloads"] = np.nan

    return row

# get_info_dataset("https://www.kaggle.com/datasets/syedanwarafridi/vehicle-sales-data")


### Iteración de datasets

In [85]:
info_datasets = []

for link in links_datasets:
    if isinstance(link,str):
        # Open link in a new tab
        driver.execute_script(f"window.open('{link}', 'new_window')")
        # Change focus
        driver.switch_to.window(driver.window_handles[-1])

        # Waiting until the page is completly load
        WebDriverWait(driver,5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".sc-eWgzEr")))

        info_datasets.append(get_info_dataset(link))
        
        print(info_datasets[-1])

        driver.close()

        # Changing focus to the main tab
        driver.switch_to.window(driver.window_handles[0])    

{'name': 'Animal Crossing New Horizons Catalog', 'url': 'https://www.kaggle.com/datasets/jessicali9530/animal-crossing-new-horizons-nookplaza-dataset', 'description': 'A comprehensive inventory of ACNH items, villagers, clothing, fish/bugs etc', 'author': 'Jessica Li ', 'votes': '42868', 'size': '(51.66 kB)', 'update_frequency': nan, 'license': nan, 'usability': nan, 'tags': ['Simulations'], 'files': '30 files', 'columns': '630 columns', 'views': '716K', 'downloads': '58.9K'}
{'name': 'Credit Card Fraud Detection', 'url': 'https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud', 'description': 'Anonymized credit card transactions labeled as fraudulent or genuine', 'author': 'Machine Learning Group - ULB and 1 collaborator', 'votes': '11083', 'size': nan, 'update_frequency': nan, 'license': nan, 'usability': nan, 'tags': ['Finance', 'Crime'], 'files': nan, 'columns': nan, 'views': nan, 'downloads': nan}
{'name': 'COVID-19 Open Research Dataset Challenge (CORD-19)', 'url': 'https://www.

In [86]:
driver.quit()

In [91]:
df_datas = pd.DataFrame(info_datasets)
df_datas

,name,url,description,author,votes,size,update_frequency,license,usability,tags,files,columns,views,downloads
0,Animal Crossing New Horizons Catalog,https://www.kaggle.com/datasets/jessicali9530/...,"A comprehensive inventory of ACNH items, villa...",Jessica Li,42868,(51.66 kB),NaN,NaN,NaN,[Simulations],30 files,630 columns,716K,58.9K
1,Credit Card Fraud Detection,https://www.kaggle.com/datasets/mlg-ulb/credit...,Anonymized credit card transactions labeled as...,Machine Learning Group - ULB and 1 collaborator,11083,NaN,NaN,NaN,NaN,"[Finance, Crime]",NaN,NaN,NaN,NaN
2,COVID-19 Open Research Dataset Challenge (CORD...,https://www.kaggle.com/datasets/allen-institut...,"An AI challenge with AI2, CZI, MSR, Georgetown...",Allen Institute For AI and 9 collaborators,10759,NaN,NaN,NaN,NaN,"[Business, Computer Science, Biology, NLP, Cor...",NaN,NaN,NaN,NaN
3,Netflix Movies and TV Shows,https://www.kaggle.com/datasets/shivamb/netfli...,Listings of movies and tv shows on Netflix - R...,Shivam Bansal,8216,NaN,NaN,NaN,NaN,"[Arts and Entertainment, Movies and TV Shows, ...",NaN,NaN,NaN,NaN
4,Chest X-Ray Images (Pneumonia),https://www.kaggle.com/datasets/paultimothymoo...,"5,863 images, 2 categories",Paul Mooney,6198,NaN,NaN,NaN,NaN,"[Health, Online Communities, Biology, Image, M...",NaN,NaN,NaN,NaN
5,Novel Corona Virus 2019 Dataset,https://www.kaggle.com/datasets/sudalairajkuma...,Day level information on covid-19 affected cases,SRK,6124,NaN,NaN,NaN,NaN,"[Health, Law, Health Conditions, Coronavirus, ...",NaN,NaN,NaN,NaN
6,Video Game Sales,https://www.kaggle.com/datasets/gregorut/video...,"Analyze sales data from more than 16,500 games.",GregorySmith,5866,NaN,NaN,NaN,NaN,"[Games, Video Games]",NaN,NaN,2.05M,546K
7,Trending YouTube Video Statistics,https://www.kaggle.com/datasets/datasnaek/yout...,Daily statistics for trending YouTube videos,Mitchell J,5344,NaN,NaN,NaN,NaN,"[Arts and Entertainment, Internet, Linguistics...",NaN,NaN,1.83M,234K
8,Data Science Cheat Sheets,https://www.kaggle.com/datasets/timoboz/data-s...,Quick help to make a data scientist's life easier,Timo Bozsolik,5001,NaN,NaN,NaN,NaN,"[Classification, Deep Learning]",NaN,NaN,393K,62.4K
9,Google Play Store Apps,https://www.kaggle.com/datasets/lava18/google-...,Web scraped data of 10k Play Store apps for an...,Lavanya,4713,NaN,NaN,NaN,NaN,"[Business, Computer Science, Internet, Video G...",NaN,NaN,1.79M,241K
